In [ ]:
import unittest
from unittest.mock import patch, Mock
import requests
from collections import Counter


class APIError(Exception):
    pass

class CatFactProcessor:
    def init(self):
        self.last_fact = ""

    def get_fact(self):
        try:
            response = requests.get("https://catfact.ninja/fact")
            response.raise_for_status()
            data = response.json()
            self.last_fact = data["fact"]
            return self.last_fact
        except requests.exceptions.RequestException as e:
            raise APIError(f"Ошибка при запросе к API: {e}") from e

    def get_fact_analysis(self):
        if not self.last_fact:
            return {"length": 0, "letter_frequencies": {}}
        fact_length = len(self.last_fact)
        letter_frequencies = dict(Counter(self.last_fact.lower()))
        return {
            "length": fact_length,
            "letter_frequencies": letter_frequencies,
        }

# --- Тесты ---

class TestCatFactProcessor(unittest.TestCase):

    @patch("requests.get")
    def test_get_fact_success(self, mock_get):
        fake_fact = "Cats sleep 70% of their lives"
        mock_response = Mock()
        mock_response.raise_for_status.return_value = None
        mock_response.json.return_value = {"fact": fake_fact}
        mock_get.return_value = mock_response

        processor = CatFactProcessor()
        result = processor.get_fact()
        self.assertEqual(result, fake_fact)
        self.assertEqual(processor.last_fact, fake_fact)

    @patch("requests.get")
    def test_get_fact_failure(self, mock_get):
        mock_get.side_effect = requests.exceptions.ConnectionError("Connection failed")

        processor = CatFactProcessor()

        with self.assertRaises(APIError) as context:
            processor.get_fact()
        self.assertIn("Ошибка при запросе к API", str(context.exception))

    def test_get_fact_analysis_empty(self):
        processor = CatFactProcessor()
        result = processor.get_fact_analysis()
        self.assertEqual(result["length"], 0)
        self.assertEqual(result["letter_frequencies"], {})

    def test_get_fact_analysis_non_empty(self):
        processor = CatFactProcessor()
        processor.last_fact = "Meow!"
        result = processor.get_fact_analysis()
        self.assertEqual(result["length"], 5)
        expected_freq = {'m': 1, 'e': 1, 'o': 1, 'w': 1, '!': 1}
        self.assertEqual(result["letter_frequencies"], expected_freq)

# Запуск тестов
if name == "main":
    unittest.main()